# Synapse Labeling (with length threshold)

In [1]:
import datajoint as dj
import numpy as np
from scipy.spatial import KDTree
import time
from tqdm import tqdm
import ipyvolume.pylab as p3


In [2]:
dj.config['display.limit'] = 20
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')

Connecting celiib@10.28.0.34:3306


In [3]:
schema = dj.schema('microns_ta3p100')

@schema
class SynapseCompartmentLabelFinalThresholdRevised(dj.Manual):
    definition = """
    -> ta3p100.Synapse
    ---
    (postsynaptic_label)->ta3p100.LabelKey
    """ # (postsynaptic_label)->ta3p100.LabelKey : tinyint unsigned

In [4]:
segment_ids = ta3p100.CoarseLabelFinal.fetch('segment_id')
mesh_fetch = (ta3p100.CleansedMesh & ta3p100.CompartmentFinal.proj()).fetch()
label_fetch = ta3p100.CoarseLabelFinal.fetch()

meshes = dict()
labels = dict()
for segment in tqdm(mesh_fetch):
    meshes[segment[1]] = (segment[5], segment[6])
    
for segment in tqdm(label_fetch):
    labels[segment[1]] = (segment[6], segment[7])
    
kdtrees = dict()
for segment_id in tqdm(segment_ids):
    kdtrees[segment_id] = KDTree(meshes[segment_id][0])
    
mesh_synapses = dict()
for segment_id in tqdm(segment_ids):
    mesh_synapses[segment_id] = np.array((ta3p100.Synapse & ta3p100.CurrentSegmentation
#                                          & dict(postsyn=segment_id)).fetch('synapse_id', 'presyn_x', 'presyn_y', 'presyn_z')).T
                                           & dict(postsyn=segment_id)).fetch('synapse_id', 'postsyn_x', 'postsyn_y', 'postsyn_z')).T

100%|██████████| 322/322 [02:32<00:00,  2.98it/s]


In [5]:
ta3p100.SynapseExclude()

segmentation segmentation id,synapse_id synapse index within the segmentation
2,6
2,80
2,162
2,237
2,451
2,473
2,595
2,629
2,638
2,716


In [6]:
matched_synapses = dict()
synapses_to_exclude = []
for segment_id in tqdm(segment_ids):
    synapses = mesh_synapses[segment_id]
    synapse_ids, postsyn_coords = synapses.T[0], synapses.T[1:].T
    kdtree = kdtrees[segment_id]
    distances, nearest_nodes = kdtree.query(postsyn_coords * [4, 4, 40])
    vertex_labels = labels[segment_id][0]
    synapse_labels = dict()
    
    for synapse_id, nearest_node, distance_node in tqdm(zip(synapse_ids, nearest_nodes,distances)):
        if distance_node > 5000:
            synapses_to_exclude.append(dict(segmentation=2,synapse_id=synapse_id))
        else:
            synapse_labels[synapse_id] = vertex_labels[nearest_node]
    matched_synapses[segment_id] = synapse_labels

  0%|          | 0/322 [00:00<?, ?it/s]
0it [00:00, ?it/s]
  0%|          | 1/322 [00:01<06:25,  1.20s/it]
0it [00:00, ?it/s]
  1%|          | 2/322 [00:01<05:28,  1.03s/it]
0it [00:00, ?it/s]
  1%|          | 3/322 [00:02<04:29,  1.18it/s]
0it [00:00, ?it/s]
  1%|          | 4/322 [00:03<05:19,  1.00s/it]
0it [00:00, ?it/s]
  2%|▏         | 5/322 [00:04<05:26,  1.03s/it]
0it [00:00, ?it/s]
  2%|▏         | 6/322 [00:05<04:48,  1.09it/s]
0it [00:00, ?it/s]
42it [00:00, 49706.76it/s]
0it [00:00, ?it/s]
  2%|▏         | 8/322 [00:06<04:01,  1.30it/s]
0it [00:00, ?it/s]
  3%|▎         | 9/322 [00:07<04:42,  1.11it/s]
0it [00:00, ?it/s]
  3%|▎         | 10/322 [00:08<04:20,  1.20it/s]
0it [00:00, ?it/s]
  3%|▎         | 11/322 [00:08<04:02,  1.28it/s]
0it [00:00, ?it/s]
  4%|▎         | 12/322 [00:10<04:44,  1.09it/s]
0it [00:00, ?it/s]
  4%|▍         | 13/322 [00:12<07:03,  1.37s/it]
0it [00:00, ?it/s]
  4%|▍         | 14/322 [00:14<08:21,  1.63s/it]
0it [00:00, ?it/s]
  5%|▍         | 15

In [7]:
print(len(synapses_to_exclude))

78345


In [28]:
#add to synapse exclude
ta3p100.SynapseExcludeFinal.insert(synapses_to_exclude,skip_duplicates=True)

In [8]:
#need to insert the synapse excludes into the table

In [9]:
ta3p100.SynapseCompartmentLabelFinalThresholdRevised()

segmentation segmentation id,synapse_id synapse index within the segmentation,postsynaptic_label numeric label of the compartment
2,2525,2
2,3848,2
2,4890,2
2,5106,2
2,7656,2
2,8128,2
2,8372,2
2,8472,2
2,9673,2
2,10597,2


In [10]:
segmentation = ta3p100.CurrentSegmentation.fetch1('segmentation')
all_synapse_labels = list()
for labeled_synapses in tqdm(matched_synapses.values()):
    segmentation_array = np.array([segmentation] * len(labeled_synapses))
    synapse_to_label = np.array(list(labeled_synapses.items())).T
    proper_label_array = np.array((segmentation_array, *synapse_to_label)).T
    all_synapse_labels.extend(proper_label_array)
all_synapse_labels_array = np.array(all_synapse_labels)

all_synapse_labels_array.T[2][all_synapse_labels_array.T[2]>12] = 1

start = time.time()
SynapseCompartmentLabelFinalThresholdRevised.insert(all_synapse_labels_array, skip_duplicates=True)
print(time.time() - start)

100%|██████████| 322/322 [00:00<00:00, 487.37it/s]


110.55493974685669


# Spine Synapse Labeler (with length threshold)

In [11]:
@schema
class SynapseSpineLabelFinalThresholdRevised(dj.Manual):
    definition = """
    # Denotes which synapses are also on Spine Heads, Spines, or Spine Necks
    -> ta3p100.SynapseCompartmentLabelFinalThresholdRevised
    ---
    (spine_label)->ta3p100.LabelKey
    """ # (spine_label)->ta3p100.LabelKey : tinyint unsigned

In [12]:
mesh_fetch = np.array((ta3p100.CleansedMesh & ta3p100.CompartmentFinal.proj()).fetch('segment_id', 'vertices', 'triangles')).T
label_fetch = np.array(ta3p100.OverlayedSpineLabelFinalRevised.fetch('segment_id', 'vertices', 'triangles')).T

meshes = dict()
for segment_id, mesh_vertices, mesh_triangles in tqdm(mesh_fetch):
    meshes[segment_id] = (mesh_vertices, mesh_triangles)

labels = dict()
for segment_id, label_vertices, label_triangles in tqdm(label_fetch):
    labels[segment_id] = (label_vertices, label_triangles)
    
kdtrees = dict()
for segment_id in tqdm(segment_ids):
    kdtrees[segment_id] = KDTree(meshes[segment_id][0])
    
mesh_synapses = dict()
for segment_id in tqdm(segment_ids):
    mesh_synapses[segment_id] = np.array((ta3p100.Synapse & ta3p100.CurrentSegmentation
                                          & dict(postsyn=segment_id)).fetch('synapse_id', 'postsyn_x','postsyn_y', 'postsyn_z')).T

100%|██████████| 322/322 [02:20<00:00,  1.77it/s]


In [13]:
matched_synapses = dict()
synapses_to_exclude_spine = []
for segment_id in tqdm(segment_ids):
    synapses = mesh_synapses[segment_id]
    synapse_ids, postsyn_coords = synapses.T[0], synapses.T[1:].T
    kdtree = kdtrees[segment_id]
    distances, nearest_nodes = kdtree.query(postsyn_coords * [4, 4, 40])
    vertex_labels = labels[segment_id][0]
    synapse_labels = dict()
    
    for synapse_id, nearest_node, distance_node in tqdm(zip(synapse_ids, nearest_nodes,distances)):
        if distance_node > 5000:
            synapses_to_exclude_spine.append(dict(segmentation=2,synapse_id=synapse_id))
        else:
            synapse_labels[synapse_id] = vertex_labels[nearest_node]
    
    matched_synapses[segment_id] = synapse_labels

  0%|          | 0/322 [00:00<?, ?it/s]
0it [00:00, ?it/s]
  0%|          | 1/322 [00:01<06:46,  1.27s/it]
0it [00:00, ?it/s]
  1%|          | 2/322 [00:01<05:49,  1.09s/it]
0it [00:00, ?it/s]
  1%|          | 3/322 [00:02<04:43,  1.12it/s]
0it [00:00, ?it/s]
  1%|          | 4/322 [00:03<05:31,  1.04s/it]
0it [00:00, ?it/s]
  2%|▏         | 5/322 [00:04<05:31,  1.05s/it]
0it [00:00, ?it/s]
  2%|▏         | 6/322 [00:05<04:52,  1.08it/s]
0it [00:00, ?it/s]
42it [00:00, 45720.42it/s]
0it [00:00, ?it/s]
  2%|▏         | 8/322 [00:06<04:02,  1.29it/s]
0it [00:00, ?it/s]
  3%|▎         | 9/322 [00:07<04:46,  1.09it/s]
0it [00:00, ?it/s]
  3%|▎         | 10/322 [00:08<04:26,  1.17it/s]
0it [00:00, ?it/s]
  3%|▎         | 11/322 [00:08<04:05,  1.26it/s]
0it [00:00, ?it/s]
  4%|▎         | 12/322 [00:10<04:56,  1.05it/s]
0it [00:00, ?it/s]
  4%|▍         | 13/322 [00:12<07:20,  1.43s/it]
0it [00:00, ?it/s]
  4%|▍         | 14/322 [00:15<08:37,  1.68s/it]
0it [00:00, ?it/s]
  5%|▍         | 15

In [14]:
print(len(synapses_to_exclude_spine))


78345


In [21]:
len(ta3p100.SynapseExclude())

72563

In [24]:
len(ta3p100.SynapseExcludeFinal()) #use this one for the future

147977

In [23]:
ta3p100.SynapseExcludeFinal.insert(synapses_to_exclude_spine,skip_duplicates=True)


In [16]:
segmentation = ta3p100.CurrentSegmentation.fetch1('segmentation')
all_synapse_labels = list()
for labeled_synapses in tqdm(matched_synapses.values()):
    segmentation_array = np.array([segmentation] * len(labeled_synapses))
    synapse_to_label = np.array(list(labeled_synapses.items())).T
    proper_label_array = np.array((segmentation_array, *synapse_to_label)).T
    all_synapse_labels.extend(proper_label_array)
all_synapse_labels_array = np.array(all_synapse_labels)

all_synapse_labels_array.T[2][all_synapse_labels_array.T[2]>15] = 1
all_synapse_labels_array = all_synapse_labels_array[all_synapse_labels_array.T[2]<=15]
all_synapse_labels_array = all_synapse_labels_array[all_synapse_labels_array.T[2]>12]

start = time.time()
SynapseSpineLabelFinalThresholdRevised.insert(all_synapse_labels_array, skip_duplicates=True)
print(time.time() - start)

100%|██████████| 322/322 [00:00<00:00, 491.88it/s]


84.82767510414124


## Visualization of Labeled Synapses (no labeled Spines yet)

In [17]:
# ta3p100.LabelKey()

In [18]:
# ta3p100.ProofreadLabel & 'segment_id=648518346341393609'

In [19]:
# label_keys = [( 0, 'not_labeled', 'no_color'), ( 1, 'label_removed', 'no_color'),
#                ( 2, 'Apical', 'blue'), ( 3, 'Basal', 'yellow'),
#                ( 4, 'Oblique', 'green'), ( 5, 'Soma', 'red'),
#                ( 6, 'Axon-Soma', 'aqua'), ( 7, 'Axon-Dendr', 'off blue'),
#                ( 8, 'Dendrite', 'purple'), ( 9, 'Distal', 'pink'),
#                (10, 'ignore Error', 'brown'), (11, 'Unlabelable', 'tan'),
#                (12, 'Cilia', 'orange'), (13, 'Spine Head (unsure if correct label)', 'red'),
#                (14, 'Spine (unsure if correct label)', 'blue'),
#                (15, 'Spine Neck (unsure if correct label)', 'green'),
#                (16, 'Bouton (unsure if correct label)', 'ignore')]
# segment_id = 648518346341393609
# synapse_label_types = np.unique(list(matched_synapses[segment_id].values()))
# label_matches = np.array(list(matched_synapses[segment_id].items())).T
# synapses_by_compartment = dict()
# synapse_centroids_by_compartment = dict()
# for synapse_label_type in synapse_label_types:
#     synapses_by_compartment[synapse_label_type] = label_matches[0][np.where(label_matches[1]==synapse_label_type)[0]]
#     centroids = np.zeros((len(synapses_by_compartment[synapse_label_type]), 3), dtype=np.uint32)
#     for i, specific_synapse in enumerate(synapses_by_compartment[synapse_label_type]):
#         centroids[i] = mesh_synapses[segment_id].T[1:].T[mesh_synapses[segment_id].T[0]==specific_synapse]
#     synapse_centroids_by_compartment[synapse_label_type] = centroids
# p3.figure(width=1024, height=1024)
# vertices, triangles = meshes[segment_id]
# p3.plot_trisurf(*vertices.T/1000, triangles=triangles, color='black')
# for label_type, label_name, label_color in label_keys:
#     if label_type in synapse_label_types:
#         print("{} -> {} : {}".format(label_type, label_name, label_color))
#         p3.scatter(*(synapse_centroids_by_compartment[label_type] * [4, 4, 40]).T/1000, color=label_color, marker='sphere', size=0.1)
# p3.squarelim()
# p3.show()